# Configuration

Define settings for remote services, create `appsettings.Development.json` using `appsettings.json` as template.

You need to manually give yourself
- `Cognitive Services OpenAI Contributor` rights on the `Azure OpenAI` resource
- `Search Index Data Contributor` right on the `Azure AI Search` resource

if you not already have it.

Also ensure to execute `azd auth login` to have access to the Azure resources.

> This example is based on [Creative Writing Assistant: Working with Agents using Prompty (Python Implementation)](https://github.com/Azure-Samples/contoso-creative-writer/tree/main).

# Install and Import Required Packages
Install and import the necessary packages using NuGet.

In [1]:
// Install the necessary packages using NuGet
#r "nuget: Azure.Identity, 1.13.1"
#r "nuget: CsvHelper, 33.0.1"
#r "nuget: Microsoft.Extensions.Configuration, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 9.0.0"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0"
#r "nuget: Microsoft.Extensions.Http.Resilience, 9.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Yaml, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.33.0-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview"

// Import the necessary libraries
using System;
using System.Threading;
using System.Net;
using System.ComponentModel;
using System.Diagnostics;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;
using System.Globalization;
using Azure.Identity;
using CsvHelper;
using Microsoft.DotNet.Interactive;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.Extensions.VectorData;
using Microsoft.Extensions.Http.Resilience;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.Chat;
using Microsoft.SemanticKernel.Agents.History;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AzureAISearch;
using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.InMemory;
using Microsoft.SemanticKernel.Data;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Plugins.Web.Bing;

Installed Packages Azure.Identity, 1.13.1 CsvHelper, 33.0.1 Microsoft.Extensions.Configuration, 9.0.0 Microsoft.Extensions.Configuration.Binder, 9.0.0 Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.0 Microsoft.Extensions.Configuration.UserSecrets, 9.0.0 Microsoft.Extensions.Http.Resilience, 9.0.0 Microsoft.SemanticKernel, 1.33.0 Microsoft.SemanticKernel.Agents.Core, 1.33.0-alpha Microsoft.SemanticKernel.Connectors.AzureAISearch, 1.33.0-preview Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.33.0 Microsoft.SemanticKernel.Connectors.InMemory, 1.33.0-preview Microsoft.SemanticKernel.Plugins.Core, 1.33.0-alpha Microsoft.SemanticKernel.Plugins.Web, 1.33.0-alpha Microsoft.SemanticKernel.Yaml, 1.33.0

# Create Kernel Builder
Create a Kernel builder instance.

In [2]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0050, SKEXP0001, SKEXP0110, SKEXP0010, OPENAI001

// NOTE: Azure AI Search or an in memory vector store can be used.
var useAzureAISearch = true;
var collectionName = "products";

// Read config settings
var configBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("./appsettings.json", optional: false)
    .AddJsonFile("./appsettings.Development.json", optional: false);   
var configuration = configBuilder.Build();

IKernelBuilder builder = Microsoft.SemanticKernel.Kernel
    .CreateBuilder()
    .AddAzureOpenAITextEmbeddingGeneration(
        configuration["EmbeddingModelDeployment"],
        configuration["Endpoint"],
        new AzureDeveloperCliCredential())
    .AddAzureOpenAIChatCompletion(
        configuration["ChatModelDeployment"],
        configuration["Endpoint"],
        new AzureDeveloperCliCredential());

if (useAzureAISearch)
{
    builder = builder.AddAzureAISearchVectorStore(
                new Uri(configuration["AzureAISearchEndpoint"]),
                new AzureDeveloperCliCredential());
}
else
{
    builder = builder.AddInMemoryVectorStore();
}

builder.Services.ConfigureHttpClientDefaults(c =>
{
    c.AddStandardResilienceHandler().Configure(o =>
    {
        o.TotalRequestTimeout.Timeout = TimeSpan.FromSeconds(120);
        o.AttemptTimeout.Timeout = TimeSpan.FromSeconds(60);
        o.CircuitBreaker.SamplingDuration = TimeSpan.FromSeconds(120);
    });
});

private sealed class FunctionInvocationFilter() : IFunctionInvocationFilter
{
    public async Task OnFunctionInvocationAsync(FunctionInvocationContext context, Func<FunctionInvocationContext, Task> next)
    {
        if (context.Function.PluginName == "SearchPlugin")
        {
            Console.WriteLine($"{context.Function.Name}:{JsonSerializer.Serialize(context.Arguments)}");
        }
        await next(context);
    }
}

builder.Services.AddSingleton<IFunctionInvocationFilter, FunctionInvocationFilter>();

// builder.Services.AddSingleton(LoggerFactory.Create(builder =>
// {
//     builder.AddConsole();
//     // NOTE: change to Trace if you want to see raw tool responses
//     builder.SetMinimumLevel(LogLevel.Debug);
// }));

Microsoft.SemanticKernel.Kernel kernel = builder.Build();

Microsoft.SemanticKernel.Kernel bingKernel = kernel.Clone();
var textSearch = new BingTextSearch(apiKey: configuration["BingAPIKey"]);
var searchPlugin = textSearch.CreateWithSearch("SearchPlugin");
bingKernel.Plugins.Add(searchPlugin);

Microsoft.SemanticKernel.Kernel vectorSearchKernel = kernel.Clone();
var vectorStore = vectorSearchKernel.Services.GetRequiredService<IVectorStore>();
var recordCollection = vectorStore.GetCollection<string, ProductDataModel>(collectionName);
await recordCollection.CreateCollectionIfNotExistsAsync();
var textEmbeddingGeneration = vectorSearchKernel.Services.GetRequiredService<ITextEmbeddingGenerationService>();
var vectorTextSearch = new VectorStoreTextSearch<ProductDataModel>(recordCollection, textEmbeddingGeneration);
var vectorSearchPlugin = vectorTextSearch.CreateWithGetTextSearchResults("SearchPlugin");
vectorSearchKernel.Plugins.Add(vectorSearchPlugin);

private sealed class ProductDataModel
{
    [VectorStoreRecordKey]
    public string Key { get; set; }

    [VectorStoreRecordData]
    [TextSearchResultName]
    public string Name { get; set; }

    [VectorStoreRecordData]
    [TextSearchResultValue]
    public string Content { get; set; }

    [VectorStoreRecordVector(1536)]
    public ReadOnlyMemory<float> Embedding { get; set; }
}

# Import sample products into vector store

In [3]:
using (var reader = new StreamReader("./../data/products.csv"))
using (var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
{
    var records = csv.GetRecords<dynamic>().ToList();

    foreach (var record in records)
    {
        var product = new ProductDataModel
        {
            Key = Guid.NewGuid().ToString(),
            Name = record.Name,
            Content = record.Content,
            Embedding = await textEmbeddingGeneration.GenerateEmbeddingAsync((string)record.Name + "" + (string)record.Content)
        };
        await recordCollection.UpsertAsync(product);
    }
}

# Create Researcher and Marketing agents, execute them

In [4]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

const string ResearcherName = "Researcher";
const string MarketingName = "Marketing";
const string WriterName = "Writer";
const string EditorName = "Editor";

string researcherYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/researcher.yaml");
ChatCompletionAgent researcherAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(researcherYaml))
    {
        Name = ResearcherName,
        Kernel = bingKernel,
        Arguments =
            new KernelArguments(
                new AzureOpenAIPromptExecutionSettings() 
                { 
                    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() 
                })
    };

string marketingYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/marketing.yaml");
ChatCompletionAgent marketingAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(marketingYaml))
    {
        Name = MarketingName,
        Kernel = vectorSearchKernel,
        Arguments =
            new KernelArguments(
                new AzureOpenAIPromptExecutionSettings() 
                { 
                    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() 
                })
    };

var researchContext = "Can you find the camping trends in 2024 and what folks are doing in this winter?";
var productContext = "Can you use a selection of tents and sleeping bags as context?";
var assignment = @"Write a fun and engaging article that includes the research and product information. 
                    The article should be between 600 and 800 words.
                    Make sure to cite sources in the article as you mention the research not at the end.";

StringBuilder sbResearchResults = new();
await foreach (ChatMessageContent response in researcherAgent.InvokeAsync(new ChatHistory(), new(){{ "research_context", researchContext } }))
{
    Console.WriteLine();
    Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
    sbResearchResults.AppendLine(response.Content);
}

StringBuilder sbProductResults = new();
await foreach (ChatMessageContent response in marketingAgent.InvokeAsync(new ChatHistory(), new(){{ "product_context", productContext } }))
{
    Console.WriteLine();
    Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
    sbProductResults.AppendLine(response.Content);
}

GetTextSearchResults:{"query":"Innovative features in modern camping tents","count":"2"}

MARKETING:
Here are the results from the searches related to tents and sleeping bags that can be used for marketing articles:

### Best Lightweight Tents for Backpacking in 2023
**1. OutdoorLiving SkyView 2-Person Tent**
- **Description**: The SkyView 2-Person Tent offers a spacious interior that accommodates two people comfortably. Made with durable waterproof materials, it provides excellent protection from the elements. Features include intuitive setup with color-coded poles, two large doors for easy access, interior pockets for organization, and two vestibules for additional storage. Mesh panels ensure good ventilation, and a rainfly offers extra weather protection. Reflective guy lines enhance nighttime visibility, and the tent packs compactly for transport. Double-stitched seams increase durability.
- **Link**: [Not Provided]

### Top-rated Sleeping Bags for Cold Weather Camping
**1. Mountai

# Creat Writer and Editor agents, put them into Group Chat

In [5]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

private sealed class NoFeedbackLeftTerminationStrategy : TerminationStrategy
{
    // Terminate when the final message contains the term "Article accepted, no further rework necessary." - all done
    protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
    {
        if(agent.Name != EditorName)
            return Task.FromResult(false);

        return Task.FromResult(history[history.Count - 1].Content?.Contains("Article accepted", StringComparison.OrdinalIgnoreCase) ?? false);
    }
}



string writerYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/writer.yaml");
ChatCompletionAgent writerAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(writerYaml))
    {
        Name = WriterName,
        Kernel = kernel,
        Arguments = new()
    };

string editorYaml = File.ReadAllText("./../ChatApp.WebApi/Agents/Prompts/editor.yaml");
ChatCompletionAgent editorAgent =
    new(KernelFunctionYaml.ToPromptTemplateConfig(editorYaml))
    {
        Name = EditorName,
        Kernel = kernel,
    };

AgentGroupChat chat =
    new(writerAgent, editorAgent)
    {
        ExecutionSettings = new AgentGroupChatSettings
        {
            SelectionStrategy = new SequentialSelectionStrategy(){ InitialAgent = writerAgent },
            TerminationStrategy = new NoFeedbackLeftTerminationStrategy()
        }
    };

Console.WriteLine("Ready!");

Ready!


# Execute Group Chat

In [ ]:
// The Agent Framework is experimental and requires warning suppression
#pragma warning disable CA2007, IDE1006, SKEXP0001, SKEXP0110, OPENAI001

writerAgent.Arguments["research_context"] = researchContext;
writerAgent.Arguments["research_results"] = sbResearchResults.ToString();
writerAgent.Arguments["product_context"] = productContext;
writerAgent.Arguments["product_results"] = sbProductResults.ToString();
writerAgent.Arguments["assignment"] = assignment;

try
{
    await foreach (ChatMessageContent response in chat.InvokeAsync())
    {
        Console.WriteLine();
        Console.WriteLine("------------------------");
        Console.WriteLine($"{response.AuthorName.ToUpperInvariant()}:{Environment.NewLine}{response.Content}");
    }
}
catch (HttpOperationException exception)
{
    Console.WriteLine(exception.Message);
    if (exception.InnerException != null)
    {
        Console.WriteLine(exception.InnerException.Message);
        if (exception.InnerException.Data.Count > 0)
        {
            Console.WriteLine(JsonSerializer.Serialize(exception.InnerException.Data, new JsonSerializerOptions() { WriteIndented = true }));
        }
    }
}